# SHAP値の計算

In [ ]:
from xgboost import XGBClassifier
from sklearn.datasets import fetch_openml  # データセットのダウンロード取得用
import pandas as pd  # データフレーム操作用
import matplotlib.pyplot as plt  # グラフ描画用
import numpy as np

In [ ]:
# 1. データセットの取得
dataset = fetch_openml("pima-indians-diabetes", version=1, as_frame=True)  # Pimaデータセットを取得
# こんな感じのデータです．8変数の説明変数．Outcomeは0 or 1
print(dataset.DESCR)

In [ ]:
# 【追加】欠損値の処理
# 欠損値の処理: Glucose，BloodPressure，SkinThickness，Insulin，BMIの5つの変数に0が入っている場合は欠損値扱いにする
# 0をNaNに置き換え
dataset.frame[["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]] = dataset.frame[
    ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
].replace(0, np.nan)
dataset.frame.head()  # データフレームの最初の5行を表示


In [ ]:
# 2. 説明変数をX，目的変数をyに分ける
# X = dataset.data
# y = dataset.target.astype("int")  # 目的変数は整数型に変換

# こっちの方が汎用性が高いです．PandasのDataFrameから変換する方法
df = dataset.frame
X = df.drop(columns=["Outcome"])  # Outcome列を除外
y = df["Outcome"].astype(int)  # Outcome列を抽出 & 整数型に変換

In [ ]:
# 3. 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split  # データ分割用
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)  # 80%訓練，20%テスト

In [ ]:
# 4. xgboostを使って学習
model = XGBClassifier(
    n_estimators=100,  # 木の数
    max_depth=3,  # 木の深さ
    learning_rate=0.1,  # 学習率
    random_state=100,  # 再現性のための乱数シード
)
model.fit(X_train, y_train)  # 訓練データで学習

## SHAP値の計算 (Tree SHAP)

In [ ]:
import shap
explainer = shap.TreeExplainer(model)  # SHAPのTreeExplainerを使用
shap_values = explainer(X_test)  # テストデータに対するSHAP値を計算
print(shap_values.shape) # (サンプル数、 変数数)

In [ ]:
shap.plots.waterfall(shap_values[0])  # 最初のサンプルに対するSHAP値のウォーターフォールプロットを表示

In [ ]:
# 特徴量"Glucose"のSHAP値をプロット
## グレーで示されているのは"Glucose"のヒストグラム
## 縦軸がSHAP値，横軸が"Glucose"の値
shap.plots.scatter(shap_values[:, "Glucose"])

In [ ]:
shap.plots.scatter(shap_values[:, "BloodPressure"])

In [ ]:
# 全てのサンプルに対するSHAP値の分布をプロット
shap.plots.beeswarm(shap_values)

In [ ]:
# 各変数の貢献度を棒グラフで表示
shap.summary_plot(shap_values, X_test, plot_type="bar")


In [ ]:
# インタラクティブにSHAP値を表示
shap.initjs() # Google Colabではこの行が必要（なはず）
shap.force_plot(explainer.expected_value, shap_values.values[:, :], X_test.iloc[:, :])

# データ個数が多すぎる場合は以下のように表示する個数をランダムに制限してください。
# N = 1000 # 表示するサンプル数
# np.random.seed(0) # 再現性のため。実行する度に同じサンプルが選ばれる
# idx = np.random.choice(X_test.shape[0], N, replace=False)  # ランダムにN個のインデックスを選択
# shap.force_plot(explainer.expected_value, shap_values.values[idx, :], X_test.iloc[idx, :])



## SHAP値の計算 (Kernel SHAP)

欠損値を許容しない他のモデルで試してみましょう

In [ ]:
# 再度データを用意
dataset = fetch_openml("pima-indians-diabetes", version=1, as_frame=True)  # Pimaデータセットを取得
dataset.frame[["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]] = dataset.frame[
    ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
].replace(0, np.nan)
df = dataset.frame


In [ ]:
# 欠損値を補完しないとモデルの学習すらできないので、まずは補完します。
# 今回は、k-近傍法を使った欠損値補完(imputation)を行います。
from sklearn.impute import KNNImputer
imputer = KNNImputer()  # k-近傍法を使って欠損値を補完
df_imputed = pd.DataFrame(
    imputer.fit_transform(df),  # 欠損値を補完
    columns=df.columns,  # 元のカラム名を保持
)  # DataFrameに戻す

# あとは同じようにtrain_test_split
X = df_imputed.drop(columns=["Outcome"])  # Outcome列を除外
y = df_imputed["Outcome"].astype(int)  # Outcome列を抽出 & 整数型に変換
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)  # 80%訓練，20%テスト

In [ ]:
# 補完前
df.head(10)

In [ ]:
# 補完後
df_imputed.head(10)

In [ ]:
# サポートベクトルマシン with カーネルトリック で学習
from sklearn.svm import SVC
model = SVC(kernel="rbf", probability=True)  # SVMのモデルを作成。probablity=Trueにして{0,1}ではなく0～1の値を出力するようにする
model.fit(X_train, y_train)  # 訓練データで学習

In [ ]:
feature_names = X_test.columns # 変数の名前一覧

def predict_fn(X):
    """予測関数"""
    _df = pd.DataFrame(X, columns=feature_names)  # NumPy配列をDataFrameに変換
    return model.predict_proba(_df)[:, 1]  # 「1」クラスの確率を返す

explainer = shap.KernelExplainer(predict_fn, X_test)  # SHAPのKernelExplainerを使用

# ※このままだとSlowだよと警告がでるがとりあえずこのまま進めます。

In [ ]:
# 計算に時間がかかるので、最初の10個のサンプルだけ計算します。
shap_values = explainer(X_test[0:10])  # テストデータに対するSHAP値を計算

In [ ]:
shap.plots.waterfall(shap_values[0])  # 最初のサンプルに対するSHAP値のウォーターフォールプロットを表示

In [ ]:
# 計算に時間がかかるので、ℎ_𝑥 (𝒛^′ )の算出時に必要な欠損値の補完につかうサンプル数を減らします。
ret = shap.kmeans(X_train, 10)  # k-meansクラスタリングを使用してSHAP値を計算 10の部分は適当に。※計算時間が許す限りは大きい数字の方がいいです
explainer = shap.KernelExplainer(predict_fn, ret)  # SHAPのKernelExplainerを使用

In [ ]:
shap_values = explainer(X_test)  # テストデータに対するSHAP値を計算

In [ ]:
shap.plots.waterfall(shap_values[0])  # 最初のサンプルに対するSHAP値のウォーターフォールプロットを表示

In [ ]:
# あとはxgboostの時と同じように
shap.plots.scatter(shap_values[:, "Glucose"])

In [ ]:
shap.plots.scatter(shap_values[:, "BloodPressure"])

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")


## 多クラス分類の場合

In [ ]:
# 1. irisデータセットを取得
dataset = fetch_openml("iris", version=1, as_frame=True)
dataset.frame.head()  # データセットの最初の5行を表示

In [ ]:
# 2. 説明変数をX，目的変数をyに分ける
df = dataset.frame
X = df.drop(columns=["class"]) 
# 目的変数は整数型に変換
class_names = df["class"].astype("category").cat.categories # 元のクラス名
df["class"] = df["class"].astype("category").cat.codes # 整数型に変換
y = df["class"]

In [ ]:
# 3. 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split  # データ分割用
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)  # 80%訓練，20%テスト
np.unique_counts(y_test)  # 訓練データのクラス分布

In [ ]:
# 4. xgboostを使って学習
model = XGBClassifier(
    n_estimators=100,  # 木の数
    max_depth=3,  # 木の深さ
    learning_rate=0.1,  # 学習率
    random_state=100,  # 再現性のための乱数シード
)
model.fit(X_train, y_train)  # 訓練データで学習

In [ ]:
import shap
explainer = shap.TreeExplainer(model)  # SHAPのTreeExplainerを使用
shap_values = explainer(X_test)  # テストデータに対するSHAP値を計算
# 多クラス識別の場合は、shap_valuesの出力が（サンプル数、変数数、クラス数）となります。
print(shap_values.shape)

In [ ]:
# 多クラス識別の場合
k = 2 # 2番目のクラスに対するSHAP値を取得
print(f"{class_names[k]}のSHAP値")
shap.plots.waterfall(shap_values[0, :, k])  # 最初のサンプルに対するSHAP値のウォーターフォールプロットを表示

In [ ]:
k = 2
print(f"{class_names[k]}のSHAP値")
shap.plots.scatter(shap_values[:, "sepallength", k])

In [ ]:
k = 2
print(f"{class_names[k]}のSHAP値")
shap.plots.beeswarm(shap_values[:, :, k])

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar", class_names=class_names)  # 特徴量の重要度を棒グラフで表示

## 回帰の場合

In [ ]:
# 1. データセットの取得 boston housing
dataset = fetch_openml("boston", version=1, as_frame=True)  # Boston Housingデータセットを取得
print(dataset.DESCR)  # データセットの説明を表示。不動産価格の予測用データセットです


In [ ]:
# CRIM: 町ごとの犯罪率
# ZN: 25,000平方フィート以上の住宅区画の割合
# INDUS: 町ごとの非小売業(工場とか)の割合
# CHAS: チャールズ川のダミー変数(1:川に接している, 0:川に接していない)
# NOX: 一酸化窒素濃度
# RM: 住居の平均部屋数
# AGE: 1940年以前に建てられた物件の割合
# DIS: ボストンの5つの雇用センターまでの距離
# RAD: 放射状高速道路へのアクセス性指標
# TAX: 10,000ドルあたりの固定資産税率
# PTRATIO: 町ごとの生徒と教師の比率
# B: 町ごとの黒人の割合
# LSTAT: 低所得者の割合
# MEDV: 住宅の中央値(目的変数)

dataset.frame.head()  # データセットの最初の5行を表示

In [ ]:
df = dataset.frame
X = df.drop(columns=["MEDV"])  # MEDV列を除外
# CHASはカテゴリー変数なので
df["CHAS"] = df["CHAS"].astype("category")  # カテゴリー型に変換
y = df["MEDV"] 

In [ ]:
# 3. 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split  # データ分割用
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)  # 80%訓練，20%テスト

In [ ]:
# 4. xgboostを使って学習
from xgboost import XGBRegressor
model = XGBRegressor(
    n_estimators=100,  # 木の数
    max_depth=3,  # 木の深さ
    learning_rate=0.1,  # 学習率
    random_state=100,  # 再現性のための乱数シード
    enable_categorical=True,  # カテゴリー変数を使用する場合はTrueにする
)
model.fit(X_train, y_train)  # 訓練データで学習

In [ ]:
import shap
explainer = shap.TreeExplainer(model)  # SHAPのTreeExplainerを使用
shap_values = explainer(X_test)  # テストデータに対するSHAP値を計算
print(shap_values.shape) # (サンプル数、 変数数)

In [ ]:
shap.plots.waterfall(shap_values[0])  # 最初のサンプルに対するSHAP値のウォーターフォールプロットを表示

In [ ]:
# 部屋数とSHAP値の関係をプロット
shap.plots.scatter(shap_values[:, "RM"])

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
# 全部表示したければ
shap.plots.beeswarm(shap_values, max_display=len(X_test.columns))  # max_displayで表示する変数の数を指定

In [ ]:
# 各変数の貢献度を棒グラフで表示
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
shap.initjs() # Google Colabではこの行が必要（なはず）
shap.force_plot(explainer.expected_value, shap_values.values[:, :], X_test.iloc[:, :])
